### <center>__画最后一年年平均sst与sss变化__

In [1]:
output                 = "/home/yyq/mls/dataprocess/warming/photo/sst_sss"
datapath               = "/data06/yyq/mls/licom2/GM-Case1501/exe/"
datapath_ctrl          = "/data06/yyq/mls/licom2/ctrl/exe/"

In [2]:
f                      = addfile(datapath+"YME.nc", "r")
f_year_ctrl            = addfile(datapath_ctrl+"YME.nc", "r")

In [3]:
sst_a                  = f->ts(:,0,:,:)
sss_a                  = f->ss(:,0,:,:)
sst_ctrl_a             = f_year_ctrl->ts(:,0,:,:)
sss_ctrl_a             = f_year_ctrl->ss(:,0,:,:)

In [4]:
lon_size               = dimsizes(sst_a(0,0,:)) - 3
lat_size               = dimsizes(sst_a(0,:,0)) - 1
sst                    = sst_a(:,:,0:lon_size)
sst_ctrl               = sst_ctrl_a(:,:,0:lon_size)
sss                    = sss_a(:,:,0:lon_size)
sss_ctrl               = sss_ctrl_a(:,:,0:lon_size)

In [5]:
;size_time              = dimsizes(sst(:,0,0)) - 1
size_time              = 29
sst_delta              = sst(size_time,:,:) - sst_ctrl(size_time,:,:)
sss_delta              = sss(size_time,:,:) - sss_ctrl(size_time,:,:)

copy_VarMeta(sst(0,:,:), sst_delta)
copy_VarMeta(sss(0,:,:), sss_delta)

printVarSummary(sss_delta)

Variable: sss_delta
Type: float
Total Size: 82800 bytes
            20700 values
Number of Dimensions: 2
Dimensions and sizes:	[lat | 115] x [lon | 180]
Coordinates: 
            lat: [90..-78]
            lon: [ 0..358]
Number Of Attributes: 6
  missing_value :	1e+35
  units :	psu
  long_name :	salinity
  lev :	-5
  time :	   1
  _FillValue :	1e+35

In [6]:
xx                     = new((/size_time + 1/),float) 
xx                     = ispan(1,size_time + 1,1)

In [8]:
sst_diff = sst(0:29,:,:) - sst_ctrl
field_sst = sst(0,:,:)
tval_sst = sst(0,:,:)
do i =0,lat_size
    do j =0,lon_size
        if (.not. ismissing(tval_sst(i,j))) then
        field_sst(i,j) = regline(xx,sst_diff(:,i,j))
        tval_sst(i,j) = field_sst@tval
        end if
    end do
end do

In [10]:
sss_diff = sss(0:29,:,:) - sss_ctrl
field_sss = sss(0,:,:)
tval_sss = sss(0,:,:)
do i =0,lat_size
    do j =0,lon_size
        if (.not. ismissing(tval_sss(i,j))) then
        field_sss(i,j) = regline(xx,sss_diff(:,i,j))
        tval_sss(i,j) = field_sss@tval
        end if
    end do
end do

In [11]:
level1                 = new((/41/),float)  
level1                 = ispan(-100,100,5)
level1                 = level1 / 100

level2                 = new((/51/),float)  
level2                 = ispan(-25,25,1)
level2                 = level2 / 100

In [12]:
wks      = gsn_open_wks("png", output)
plots    = new(2,"graphic")

res_t = True
    res_t@gsnFrame         = False
    res_t@gsnDraw          = False
    res_t@cnLinesOn        = False
    res_t@cnLineLabelsOn   = False
    res_t@cnFillOn         = False
    res_t@cnInfoLabelOn    = False
    res_t@gsnLeftString = ""
    res_t@gsnRightString = ""
    res_t@cnFillOn             = True             ; fill contour intervals
    res_t@cnLevelSelectionMode = "ExplicitLevels"
    res_t@lbLabelBarOn         = False
    res_t@cnMonoFillPattern    = False            ; want multiple patterns 
    res_t@cnFillPatterns       = (/17,-1,17/)     ; the patterns
    res_t@cnMonoFillScale      = False            ; want different densities 
    res_t@cnFillScales         = (/1.,1,1./)      ; the densities
    res_t@cnFillColors         = (/1,3,1/)
    res_t@cnFillDotSizeF       = 0.0025            ; increase dot size of contours
    tvall=cdft_t(0.005,28)
    res_t@cnLevels             = (/tvall,-tvall/) ;

res_0    = True
    res_0@gsnDraw                   =  False
    res_0@gsnFrame                  =  False
    res_0@cnFillOn                  =  True
    res_0@cnInfoLabelOn             =  False
    res_0@cnLinesOn                 =  False
    res_0@cnLineLabelsOn            =  False
    res_0@lbLabelBarOn              =  False
    res_0@cnLevelSelectionMode      = "ExplicitLevels"
    res_0@cnLevels                  = level1
    res_0@gsnLeftString             = "delta sst"
    res_0@gsnLeftStringFontHeightF  = 15
    res_0@gsnRightString            = "K"
    res_0@gsnRightStringFontHeightF = 15
    res_0@cnFillPalette             = "BlRe"
    res_0@mpCenterLonF              = 180
    res_0@lbLabelBarOn              = True
    res_0@lbLabelStride              = 5                 ;colorbar每5个色块一个标注
    res_0@lbLabelAutoStride          = False             ;colorbar每条等值线均标注
    res_0@lbBoxSeparatorLinesOn      = False             ;colorbar内部边框隐藏
    res_0@lbBoxLinesOn               = False             ;colorbar外边框设置为隐藏
    res_0@pmLabelBarHeightF          = 0.05              ;colorbar高度
    res_0@pmLabelBarWidthF           = 0.8               ;colorbar宽度
    res_0@pmLabelBarOrthogonalPosF   = 0.1               ;colorbar水平偏移
    res_0@lbBoxEndCapStyle           = "TriangleBothEnds"
plots(0) = gsn_csm_contour_map(wks,sst_delta, res_0)
plot_test_sst  = gsn_csm_contour(wks,tval_sst,res_t)
overlay(plots(0), plot_test_sst)

res_1 = True
    res_1@gsnDraw                   =  False
    res_1@gsnFrame                  =  False
    res_1@cnFillOn                  =  True
    res_1@cnInfoLabelOn             =  False
    res_1@cnLinesOn                 =  False
    res_1@cnLineLabelsOn            =  False
    res_1@lbLabelBarOn              =  False
    res_1@cnFillPalette             = "BlRe"
    res_1@cnLevelSelectionMode      = "ExplicitLevels"
    res_1@cnLevels                  = level2
    res_1@gsnLeftString             = "delta salinity"
    res_1@gsnLeftStringFontHeightF  = 15
    res_1@mpCenterLonF              = 180
    res_1@lbLabelBarOn              = True
    res_1@lbLabelStride              = 5                 ;colorbar每5个色块一个标注
    res_1@lbLabelAutoStride          = False             ;colorbar每条等值线均标注
    res_1@lbBoxSeparatorLinesOn      = False             ;colorbar内部边框隐藏
    res_1@lbBoxLinesOn               = False             ;colorbar外边框设置为隐藏
    res_1@pmLabelBarHeightF          = 0.05              ;colorbar高度
    res_1@pmLabelBarWidthF           = 0.8               ;colorbar宽度
    res_1@pmLabelBarOrthogonalPosF   = 0.1               ;colorbar水平偏移
    res_1@lbBoxEndCapStyle           = "TriangleBothEnds"
plots(1) = gsn_csm_contour_map(wks,sss_delta, res_1)
plot_test_sss  = gsn_csm_contour(wks,tval_sss,res_t)
overlay(plots(1), plot_test_sss)

resP    = True
gsn_panel(wks, plots, (/1,2/), resP)

.git      addwater  grid      test      tool      warming.git      addwater  grid      test      tool      warming

![delta_sst](../photo/sst_sss.png)